Make API call of Guardian to return all Jay Rayner articles. First query the API for the list of articles - this returns a page of results. We fetch the first batch of links to follow here, then we need to work through the rest of the list of pages, getting all those links.  

In [19]:
import requests
import re 
import os

page_size = 15
page = 1
api_key = os.environ.get('GUARDIAN_API_KEY')

#base url which looks for the jay tags 
url = f"https://content.guardianapis.com/search?api-key={api_key}&tag=food/series/jay-rayner-on-restaurants&page-size={page_size}"
payload = {}
headers = {
  'Cookie': 'AWSELB=75B9BD811C5C032EDEF76366759629DCCB8726D7A371904BEC1C3B7DFC40019571E370E2C4E4519DDF3CD336789F71716B110728D88A7C69AE901D39C1821FF2C5E227F5F9; AWSELBCORS=75B9BD811C5C032EDEF76366759629DCCB8726D7A371904BEC1C3B7DFC40019571E370E2C4E4519DDF3CD336789F71716B110728D88A7C69AE901D39C1821FF2C5E227F5F9; BCSI-CS-682e9ee7ab0fdcff=1'
}

#function to go through the results
def review_grabber(apiresults, count):
    with open('jaymap.tsv', 'a') as csvfile:
        #if status is ok, work through the list of results and find the postcodes in the linked articles
        if apiresults['response']['status'] == 'ok':
            for result in apiresults['response']['results']:
                #print(f"visiting {result['webUrl']}, {count}, {len(apiresults)}")
                article = requests.request("GET", result['webUrl'], headers=headers, data = payload)
                postcode = re.findall(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b', article.text)
                #if there is a postcode print it, to avoid out of bounds error caught in if loop
                if len(postcode) > 0:
                    #output as TSV as there are commas in some of the titles
                    csvfile.write(f"\n{result['webTitle']}\t{result['webUrl']}\t{postcode[0]}\t{result['webPublicationDate']}")
                    print(count, result['webTitle'], result['webUrl'], postcode[0], result['webPublicationDate'])
                else:
                    csvfile.write(f"\n{result['webTitle']}\t{result['webUrl']}\t{result['webPublicationDate']}")
                    print(count, result['webTitle'], result['webUrl'], result['webPublicationDate'])
                count += 1 
            return count

#fetch the first page listing articles by jay
response = requests.request("GET", url, headers=headers, data = payload)
responsejson = response.json()
#how many pages are there in the response, as we will need to get them all
#print(responsejson['response'])
pages = responsejson['response']['pages']
#total articles to check off later
total_articles = responsejson['response']['total']
print(f"there are {pages} pages of articles to get, {page_size} articles on each page, {total_articles} articles in total")
#send the whole response into the function to visit the page and get the postcode
count = 0 
count = review_grabber(responsejson, count)

there are 72 pages of articles to get, 15 articles on each page, 1073 articles in total
visiting https://www.theguardian.com/food/2021/aug/29/jay-rayner-restaurant-review-the-gpo-liverpool-could-be-first-class-when-its-sorted, 0, 1
0 The GPO, Liverpool: ‘They’re trying to work all the angles’ – restaurant review https://www.theguardian.com/food/2021/aug/29/jay-rayner-restaurant-review-the-gpo-liverpool-could-be-first-class-when-its-sorted L1 6DA 2021-08-29T05:00:09Z
visiting https://www.theguardian.com/food/2021/aug/22/kurisu-omakase-at-ichiban-sushi-london-it-is-both-dinner-and-theatre-restaurant-review, 1, 1
1 Kurisu Omakase at Ichiban Sushi, London: ‘It is both dinner and theatre’ – restaurant review https://www.theguardian.com/food/2021/aug/22/kurisu-omakase-at-ichiban-sushi-london-it-is-both-dinner-and-theatre-restaurant-review SW9 8PY 2021-08-22T07:23:49Z
visiting https://www.theguardian.com/food/2021/aug/15/the-polo-lounge-at-the-dorchester-hotel-dismal-food-at-eye-popping-price

In [20]:
#work through the rest of the pages getting the links
page = 2
while page <= pages:
    print(page)
    url = f"https://content.guardianapis.com/search?api-key={api_key}&tag=food/series/jay-rayner-on-restaurants&page-size={page_size}&page={page}"
    response2 = requests.request("GET", url, headers=headers, data = payload)
    response2json = response2.json()
    count = review_grabber(response2json, count)
    page += 1

2
visiting https://www.theguardian.com/food/2021/apr/25/jay-rayner-restaurant-review-i-love-the-freestyle-vibe-at-brat-at-climpsons-arch, 15, 1
15 Brat at Climpson’s Arch, London: ‘I love the rackety, freestyle vibe’: restaurant review https://www.theguardian.com/food/2021/apr/25/jay-rayner-restaurant-review-i-love-the-freestyle-vibe-at-brat-at-climpsons-arch E8 3SB 2021-04-25T05:00:08Z
visiting https://www.theguardian.com/food/2021/apr/18/jay-rayner-on-restaurants-what-i-learnt-in-my-year-of-lockdown-stuck-at-home, 16, 1
16 Cookbooks are great, pastry is easy… https://www.theguardian.com/food/2021/apr/18/jay-rayner-on-restaurants-what-i-learnt-in-my-year-of-lockdown-stuck-at-home 2021-04-18T05:00:55Z
visiting https://www.theguardian.com/food/2021/apr/11/the-old-scrapbook-collections-that-tell-the-story-of-our-lives-recipes, 17, 1
17 The old scrapbook recipe collections that tell the story of our lives https://www.theguardian.com/food/2021/apr/11/the-old-scrapbook-collections-that-tell

In [18]:
page_size = 5
page = 2
api_key = os.environ.get('GUARDIAN_API_KEY')

#base url which looks for the jay tags 
url = f"https://content.guardianapis.com/search?api-key={api_key}&tag=food/series/jay-rayner-on-restaurants&page-size={page_size}&page={page}"
payload = {}
headers = {
  'Cookie': 'AWSELB=75B9BD811C5C032EDEF76366759629DCCB8726D7A371904BEC1C3B7DFC40019571E370E2C4E4519DDF3CD336789F71716B110728D88A7C69AE901D39C1821FF2C5E227F5F9; AWSELBCORS=75B9BD811C5C032EDEF76366759629DCCB8726D7A371904BEC1C3B7DFC40019571E370E2C4E4519DDF3CD336789F71716B110728D88A7C69AE901D39C1821FF2C5E227F5F9; BCSI-CS-682e9ee7ab0fdcff=1'
}
count = 1
#function to go through the results
def review_grabber(apiresults, count):
    with open('jaymap_new.csv', 'a') as csvfile:
        #if status is ok, work through the list of results and find the postcodes in the linked articles
        if apiresults['response']['status'] == 'ok':
            for result in apiresults['response']['results']:
                article = requests.request("GET", result['webUrl'], headers=headers, data = payload)
                postcode = re.findall(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b', article.text)
                if len(postcode) > 0:
                    csvfile.write(f"\n{count},{result['webTitle']},{result['webUrl']},{postcode[0]}")
                    print(count, result['webTitle'], result['webUrl'], postcode[0])
                count += 1
            return count

#fetch the first page listing articles by jay
response = requests.request("GET", url, headers=headers, data = payload)
responsejson = response.json()
responsejson

{'response': {'status': 'ok',
  'userTier': 'developer',
  'total': 1073,
  'startIndex': 6,
  'pageSize': 5,
  'currentPage': 2,
  'pages': 215,
  'orderBy': 'newest',
  'results': [{'id': 'food/2021/jul/25/jay-rayner-restaurant-review-sonny-stores-bristol-they-get-things-absolutely-right',
    'type': 'article',
    'sectionId': 'food',
    'sectionName': 'Food',
    'webPublicationDate': '2021-07-25T05:00:43Z',
    'webTitle': 'Sonny Stores, Bristol: ‘They get things absolutely right’ – restaurant review',
    'webUrl': 'https://www.theguardian.com/food/2021/jul/25/jay-rayner-restaurant-review-sonny-stores-bristol-they-get-things-absolutely-right',
    'apiUrl': 'https://content.guardianapis.com/food/2021/jul/25/jay-rayner-restaurant-review-sonny-stores-bristol-they-get-things-absolutely-right',
    'isHosted': False,
    'pillarId': 'pillar/lifestyle',
    'pillarName': 'Lifestyle'},
   {'id': 'food/2021/jul/18/jay-rayner-restaurant-review-the-double-red-duke-oxfordshire-it-is-a-cl

In [ ]:

#how many pages are there in the response, as we will need to get them all
pages = responsejson['response']['pages']
#total articles to check off later
total_articles = responsejson['response']['total']
print(f"there are {pages} pages of articles to get, {page_size} articles on each page, {total_articles} articles in total")
#send the whole response into the function to visit the page and get the postcode